In [1]:
import pandas as pd
import multiprocessing
import pickle
import numpy as np
import difflib
import regex as re

from math import sqrt
from time import time
from sklearn.cluster import DBSCAN, AgglomerativeClustering, KMeans
from sklearn.neighbors import NearestNeighbors
from gensim.models import Word2Vec
from kneed import KneeLocator

In [2]:
pd.set_option('max_colwidth', 1600)

In [3]:
cpu_number = multiprocessing.cpu_count()

w2v_window= 7

In [4]:
solver_total_errors_df= pd.read_csv('solver-error-clean-data.csv')

with open("solver-errors-clean-clustering-data.txt", "rb") as fp:
    clean_clustering_data = pickle.load(fp)

In [33]:
def detect_embedding_size(tokens):
    flat_list = [item for row in tokens for item in row]
    vocab = set(flat_list)
    embedding_size = round(len(vocab) ** (2/3))
    if embedding_size >= 400:
        embedding_size = 400
    return embedding_size

w2v_size = detect_embedding_size(clean_clustering_data)

In [6]:
def tokens_vectorization(clustering_data, w2v_size, w2v_window, cpu_number, model_name):
    iterations = 100
    word2vec = Word2Vec(clustering_data,
                           size = w2v_size, 
                           window = w2v_window, 
                           min_count=1, 
                           workers = cpu_number,
                           iter=iterations)
    word2vec.save(model_name)
    return word2vec

In [7]:
word2vec = tokens_vectorization(clean_clustering_data, 
                                 w2v_size = w2v_size, 
                                 w2v_window= w2v_window, 
                                 cpu_number = cpu_number, 
                                 model_name='word2vec.model')

In [8]:
def sentence_vectorization(clustering_data, word2vec):
    sent2vec = []
    for sent in clustering_data:
        sent_vec = []
        numw = 0
        for w in sent:
            try:
                sent_vec = word2vec[w] if numw == 0 else np.add(sent_vec, word2vec[w])
                numw += 1
            except Exception:
                pass
        sent2vec.append(np.asarray(sent_vec) / numw)   
    return np.vstack(sent2vec)

In [9]:
sent2vec = sentence_vectorization(clean_clustering_data, word2vec)

/home/skotak/.local/share/virtualenvs/solver-error-classifier-SslAmjTI/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [10]:
sent2vec.shape

(1624, 19)

In [11]:
def kneighbors(sent2vec):
    k = round(sqrt(len(sent2vec)))
    neigh = NearestNeighbors(n_neighbors=k)
    nbrs = neigh.fit(sent2vec)
    distances, indices = nbrs.kneighbors(sent2vec)
    distances = [np.mean(d) for d in np.sort(distances, axis=0)]
    return distances

avg_distances = kneighbors(sent2vec)

In [12]:
def epsilon_search(distances):
    kneedle = KneeLocator(distances, list(range(len(distances))))
    epsilon = max(kneedle.all_elbows) if (len(kneedle.all_elbows) > 0) else 1
    return epsilon


def hierarchical(epsilon, sent2vec):
        cluster_labels = AgglomerativeClustering(n_clusters=None,linkage='ward',
                                                 distance_threshold=epsilon).fit_predict(sent2vec)
        return cluster_labels

In [13]:
epsilon = epsilon_search(avg_distances)
cluster_labels = hierarchical(epsilon, sent2vec)

In [14]:
cluster_labels

array([29, 10, 24, ..., 12,  1, 29])

In [15]:
solver_total_errors_df['cluster_no.'] = cluster_labels

In [16]:
solver_total_errors_df.head(10)

,package_name,package_version,command,message,return_code,stderr,stdout,Error_info,command_info,cwd,Complete_output,ERROR,Exception,specific_error,clustering_data,clean_clustering_data,cluster_no.
0,tryton,3.2.20,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps tryton==3.2.20 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org","Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-_gczw75i/tryton/setup.py'""'""'; __file__='""'""'/tmp/pip-install-_gczw75i/tryton/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-_gczw75i/tryton/pip-egg-info\n cwd: /tmp/pip-install-_gczw75i/tryton/\n Complete output (6 lines):\n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-_gczw75i/tryton/setup.py"", line 289\n print ""makensis.exe not found: installers can not be created, ""\\n ^\n SyntaxError: Missing parentheses in call to 'print'. Did you mean print(""makensis.exe not found: installers can not be created, ""\)?\n ----------------------------------------\nERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.\n",1.0,"ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-_gczw75i/tryton/setup.py'""'""'; __file__='""'""'/tmp/pip-install-_gczw75i/tryton/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-_gczw75i/tryton/pip-egg-info\n cwd: /tmp/pip-install-_gczw75i/tryton/\n Complete output (6 lines):\n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-_gczw75i/tryton/setup.py"", line 289\n print ""makensis.exe not found: installers can not be created, ""\\n ^\n SyntaxError: Missing parentheses in call to 'print'. Did you mean print(""makensis.exe not found: installers can not be created, ""\)?\n ----------------------------------------\nERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.\n",Collecting tryton==3.2.20\n Downloading https://files.pythonhosted.org/packages/8d/b6/a88e6e4df1557cf3f265b12fd585f36e212fd8669aa394c9fbb5a86e56d3/tryton-3.2.20.tar.gz (811kB)\n,Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:,"['command: /home/solver/venv/bin/python3 -c \'import sys, setuptools, tokenize; sys.argv[0] = \'""\'""\'/tmp/pip-install-_gczw75i/tryton/setup.py\'""\'""\'; __file__=\'""\'""\'/tmp/pip-install-_gczw75i/tryton/setup.py\'""\'""\';f=getattr(tokenize, \'""\'""\'open\'""\'""\', open)(__file__);code=f.read().replace(\'""\'""\'\\r\\n\'""\'""\', \'""\'""\'\\n\'""\'""\');f.close();exec(compile(code, __file__, \'""\'""\'exec\'""\'""\'))\' egg_info --egg-base /tmp/pip-install-_gczw75i/tryton/pip-egg-info']",['cwd: /tmp/pip-install-_gczw75i/tryton/'],"['Complete output (6 lines):', 'Traceback (most recent call last):', 'File ""<string>"", line 1, in <module>', 'File ""/tmp/pip-install-_gczw75i/tryton/setup.py"", line 289', 'print ""makensis.exe not found: installers can not be created, ""\\', '^', 'SyntaxError: Missing parentheses in call to \'print\'. Did you mean print(""makensis.exe not found: installers can not be created, ""\\)?']",['ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.'],NaN,"['SyntaxError: Missing parentheses in call to \'print\'. Did you

In [17]:
def clustered_output(solver_total_errors_df, mode='INDEX'):
    groups = {}
    for key, value in solver_total_errors_df.groupby(['cluster_no.']):
        if mode == 'ALL':
            groups[str(key)] = value.to_dict(orient='records')
        elif mode == 'INDEX':
            groups[str(key)] = value.index.values.tolist()
        elif mode == 'TARGET':
            groups[str(key)] = value[self.target].values.tolist()
        elif mode == 'CLEANED':
            groups[str(key)] = value['clean_clustering_data'].values.tolist()
    return groups

In [18]:
def find_matching_blocks(strings):
    curr = strings[0]
    if len(strings) == 1:
        return curr
    else:
        cnt = 1
        for i in range(cnt, len(strings)):
            matches = difflib.SequenceMatcher(None, curr, strings[i])
            common = []
            for match in matches.get_matching_blocks():
                common.append(curr[match.a:match.a + match.size])
            #curr = ''.join(common)
            curr = ','.join(str(v) for v in common)
            cnt = cnt + 1
            if cnt == len(strings):
                break
        if curr == '':
            'NO COMMON PATTERNS HAVE BEEN FOUND'
        return curr

def get_similarity(rows):
    s = []
    for i in range(0, len(rows)):
        s.append(difflib.SequenceMatcher(None, rows[0], rows[i]).ratio() * 100)
    return s

In [19]:
STATISTICS = ["cluster_name", "cluster_size", "pattern",
              "mean_length", "mean_similarity", "std_length", "std_similarity"]

def statistics(solver_total_errors_df, output_mode='frame'):
    """
    Returns dictionary with statistic for all clusters
    "cluster_name" - name of a cluster
    "cluster_size" = number of log messages in cluster
    "pattern" - all common substrings in messages in the cluster
    "vocab" - vocabulary of all messages within the cluster (without punctuation and stop words)
    "vocab_length" - the length of vocabulary
    "mean_length" - average length of log messages in cluster
    "std_length" - standard deviation of length of log messages in cluster
    "mean_similarity" - average similarity of log messages in cluster
    (calculated as the levenshtein distances between the 1st and all other log messages)
    "std_similarity" - standard deviation of similarity of log messages in cluster
    :param clustered_df:
    :param output_mode: frame | dict
    :return:
    """
    clusters = []
    clustered_df = clustered_output(solver_total_errors_df, mode='CLEANED')
    for item in clustered_df:
        row = clustered_df[item]
        matcher = find_matching_blocks(row)
        lengths = [len(s) for s in row]
        similarity = get_similarity(row)
        clusters.append([item,
                         len(row),
                         matcher,
                         np.mean(lengths),
                         np.mean(similarity),
                         np.std(lengths) if len(row) > 1 else 0,
                         np.std(similarity)])
    df = pd.DataFrame(clusters, columns=STATISTICS).round(2).sort_values(by='cluster_size', ascending=False)
    if output_mode == 'frame':
        return df
    else:
        return df.to_dict(orient='records')

In [20]:
stat = statistics(solver_total_errors_df, output_mode='frame')

In [21]:
stat_df = pd.DataFrame.from_dict(stat)
len(stat_df)

41

In [22]:
stat_df.sort_values(by='cluster_size', ascending=False)

,cluster_name,cluster_size,pattern,mean_length,mean_similarity,std_length,std_similarity
4,4,365,"['No', 'matching', 'distribution', 'found'],",43.0,100.0,0.0,0.0
29,29,320,"['SyntaxError'],",15.0,100.0,0.0,0.0
1,1,284,"['Failed', 'building', 'wheel'],",31.0,100.0,0.0,0.0
16,16,178,"['ModuleNotFoundError'],",23.0,100.0,0.0,0.0
10,10,144,"['FileNotFoundError'],",21.0,100.0,0.0,0.0
20,20,69,"['NameError'],",13.0,100.0,0.0,0.0
0,0,60,"['Command', 'errored', 'out', 'with', 'exit', 'status', 'python', 'egg', 'info', 'Check', 'the', 'logs'],",104.0,100.0,0.0,0.0
13,13,33,"['AttributeError'],",18.0,100.0,0.0,0.0
6,6,30,"['TypeError'],",13.0,100.0,0.0,0.0
24,24,23,"['RuntimeError'],",16.0,100.0,0.0,0.0


In [23]:
def in_cluster(all_cluster_labels, cluster_label):
    results = []
    for idx, l in enumerate(all_cluster_labels):
        if l == cluster_label:
            results.append(solver_total_errors_df['clean_clustering_data'].values[idx])
    return results

In [24]:
in_cluster(solver_total_errors_df['cluster_no.'], 16)

["['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundError']",
 "['ModuleNotFoundEr

In [27]:
solver_total_errors_df['CLASS']= solver_total_errors_df['cluster_no.'].map(stat_df['pattern'])

In [31]:
solver_total_errors_df.head(20)

,package_name,package_version,command,message,return_code,stderr,stdout,Error_info,command_info,cwd,Complete_output,ERROR,Exception,specific_error,clustering_data,clean_clustering_data,cluster_no.,CLASS
0,tryton,3.2.20,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps tryton==3.2.20 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org","Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-_gczw75i/tryton/setup.py'""'""'; __file__='""'""'/tmp/pip-install-_gczw75i/tryton/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-_gczw75i/tryton/pip-egg-info\n cwd: /tmp/pip-install-_gczw75i/tryton/\n Complete output (6 lines):\n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-_gczw75i/tryton/setup.py"", line 289\n print ""makensis.exe not found: installers can not be created, ""\\n ^\n SyntaxError: Missing parentheses in call to 'print'. Did you mean print(""makensis.exe not found: installers can not be created, ""\)?\n ----------------------------------------\nERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.\n",1.0,"ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-_gczw75i/tryton/setup.py'""'""'; __file__='""'""'/tmp/pip-install-_gczw75i/tryton/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' egg_info --egg-base /tmp/pip-install-_gczw75i/tryton/pip-egg-info\n cwd: /tmp/pip-install-_gczw75i/tryton/\n Complete output (6 lines):\n Traceback (most recent call last):\n File ""<string>"", line 1, in <module>\n File ""/tmp/pip-install-_gczw75i/tryton/setup.py"", line 289\n print ""makensis.exe not found: installers can not be created, ""\\n ^\n SyntaxError: Missing parentheses in call to 'print'. Did you mean print(""makensis.exe not found: installers can not be created, ""\)?\n ----------------------------------------\nERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.\n",Collecting tryton==3.2.20\n Downloading https://files.pythonhosted.org/packages/8d/b6/a88e6e4df1557cf3f265b12fd585f36e212fd8669aa394c9fbb5a86e56d3/tryton-3.2.20.tar.gz (811kB)\n,Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:,"['command: /home/solver/venv/bin/python3 -c \'import sys, setuptools, tokenize; sys.argv[0] = \'""\'""\'/tmp/pip-install-_gczw75i/tryton/setup.py\'""\'""\'; __file__=\'""\'""\'/tmp/pip-install-_gczw75i/tryton/setup.py\'""\'""\';f=getattr(tokenize, \'""\'""\'open\'""\'""\', open)(__file__);code=f.read().replace(\'""\'""\'\\r\\n\'""\'""\', \'""\'""\'\\n\'""\'""\');f.close();exec(compile(code, __file__, \'""\'""\'exec\'""\'""\'))\' egg_info --egg-base /tmp/pip-install-_gczw75i/tryton/pip-egg-info']",['cwd: /tmp/pip-install-_gczw75i/tryton/'],"['Complete output (6 lines):', 'Traceback (most recent call last):', 'File ""<string>"", line 1, in <module>', 'File ""/tmp/pip-install-_gczw75i/tryton/setup.py"", line 289', 'print ""makensis.exe not found: installers can not be created, ""\\', '^', 'SyntaxError: Missing parentheses in call to \'print\'. Did you mean print(""makensis.exe not found: installers can not be created, ""\\)?']",['ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.'],NaN,"['SyntaxError: Missing parentheses in call to \'print\'. D

In [29]:
solver_total_errors_df.columns

Index(['package_name', 'package_version', 'command', 'message', 'return_code',
       'stderr', 'stdout', 'Error_info', 'command_info', 'cwd',
       'Complete_output', 'ERROR', 'Exception', 'specific_error',
       'clustering_data', 'clean_clustering_data', 'cluster_no.', 'CLASS'],
      dtype='object')